In [1]:
%matplotlib inline


import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import psycopg2

sys.path.append('..')
from scripts import util
import folium

In [2]:
conn = util.connect_rds()

In [3]:
def clean_euro_decimals(series):
    '''
    Cleans the euro , decimals (ugh) into point decimals
    and converts series into numeric
    '''
    return pd.to_numeric(series.str.replace(',','.'))

In [4]:
# import loop locations that are associated with location 8484
sql_loc = "select startlocatiefordisplaylat as lat, startlocatiefordisplaylong as lon \
from rws_schema.speed \
where specificlocation = 8484 and specificlane = 'lane1' \
order by random();"
df_loc = pd.read_sql(sql_loc, con=conn)

In [19]:
df_loc.head()

,lat,lon
0,52.2275,5.4567
1,52.2332,5.4604
2,52.2275,5.4567
3,52.2275,5.4567
4,52.2332,5.4604


In [6]:
# create map to plot loop locations
map_loops = folium.Map(location=[52.2275,5.45671],tiles='Stamen Toner')
lat_loops = [row for row in df_loc['lat']]
lon_loops = [row for row in df_loc['lon']]

In [20]:
# add loop locations to map (in blue)
for i in range(10):

        folium.CircleMarker([lat_loops[i], lon_loops[i]], \
                  popup=''+str(i), radius=3.0).add_to(map_loops)
   


In [22]:
# import all hectopunten located along A28
sql_hect = "select hecto,coords_x2 as lat, coords_x1 as lon \
from rws_schema.hectopunten \
where routeltr = 'A' and routenr = '28' \
order by random();"
df_hect = pd.read_sql(sql_hect, con=conn)

df_hect['lat'] = clean_euro_decimals(df_hect['lat'])
df_hect['lon'] = clean_euro_decimals(df_hect['lon'])

In [10]:
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from shapely.ops import nearest_points

In [11]:
# create geoframe using hectopunten points
geometry = [Point(xy) for xy in zip(df_hect["lon"], df_hect["lat"])]
crs = {'init': 'epsg:4326'} #Coordinate reference system
gdf = GeoDataFrame(df_hect, crs=crs, geometry=geometry)

In [12]:
def find_nearest_hectopoint(input_lat:float, input_lon:float , gdf):
    "Return coords of nearest hectopoint"
    input_point = Point(input_lat,input_lon)
    nearest = gdf.geometry == nearest_points(input_point,gdf.unary_union)[1]
    hecto_val = gdf[nearest]["hecto"].get_values()[0]
    return hecto_val

In [23]:
# for each speed/flow datapoint, find the nearest hectopunten and add it to the map (in pink)
for i in range(10):
        hecto_pt = find_nearest_hectopoint(df_loc['lat'][i], df_loc['lon'][i], gdf)
        temp = df_hect[(df_hect['hecto']== hecto_pt)].values[0]
        folium.CircleMarker([temp[1], temp[2]], popup=''+str(i), radius=3.0, \
                  fill=True, fill_color='#ff346d', color = '#ff346d')\
        .add_to(map_loops)

TypeError: must be real number, not str

In [15]:
map_loops

In [17]:
# Generate plot of the hectopoints interpreted 
map_hectopts = folium.Map(location=[52.2275,5.45671],tiles='Stamen Toner')
for i in range(1000):

        folium.CircleMarker([df_hect.lat[i], df_hect.lon[i]], \
                  popup=''+str(i), radius=3.0).add_to(map_hectopts)
map_hectopts